In [0]:
## Libraries Prequisities
!pip install scikit-learn
!pip install numpy==1.16.1

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [1]:
import os
os.chdir("/content/drive/")
!ls

'My Drive'


In [0]:
import os
os.chdir("My Drive/Computer_Vision_Masters/Wearable_Sensors_Code/")

In [0]:
## Libraries
import numpy as np
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.metrics.classification import accuracy_score, recall_score, f1_score
import scipy.stats as st
import sys
import warnings
warnings.filterwarnings('ignore')
np.random.seed(12227)

In [0]:
def A(sample):
    feat = []
    for col in range(0,sample.shape[1]):
        average = np.average(sample[:,col])
        feat.append(average)

    return feat

def SD(sample):
    feat = []
    for col in range(0, sample.shape[1]):
        std = np.std(sample[:, col])
        feat.append(std)

    return feat

def AAD(sample):
    feat = []
    for col in range(0, sample.shape[1]):
        data = sample[:, col]
        add = np.mean(np.absolute(data - np.mean(data)))
        feat.append(add)

    return feat

def ARA(sample):
    #Average Resultant Acceleration[1]:
    # Average of the square roots of the sum of the values of each axis squared √(xi^2 + yi^2+ zi^2) over the ED
    feat = []
    sum_square = 0
    sample = np.power(sample, 2)
    for col in range(0, sample.shape[1]):
        sum_square = sum_square + sample[:, col]

    sample = np.sqrt(sum_square)
    average = np.average(sample)
    feat.append(average)
    return feat

def TBP(sample):
    from scipy import signal
    feat = []
    sum_of_time = 0
    for col in range(0, sample.shape[1]):
        data = sample[:, col]
        peaks = signal.find_peaks_cwt(data, np.arange(1,4))

        feat.append(peaks)

    return feat

In [0]:
## Feature Extraction

def feature_extraction(X):
    # Extracts the features, as mentioned by Catal et al. 2015
    # Average - A,
    # Standard Deviation - SD,
    # Average Absolute Difference - AAD,
    # Average Resultant Acceleration - ARA(1),
    # Time Between Peaks - TBP
    X_tmp = []
    for sample in X:
        features = A(sample)
        features = np.hstack((features, A(sample)))
        features = np.hstack((features, SD(sample)))
        features = np.hstack((features, AAD(sample)))
        features = np.hstack((features, ARA(sample)))
        #features = np.hstack((features, TBP(sample)))
        X_tmp.append(features)

    X = np.array(X_tmp)
    return X

In [0]:
def train_logistic_regression():
    from sklearn.linear_model import LogisticRegression
    clf = LogisticRegression(multi_class='ovr')
    return clf

In [0]:
def train_j48():
    from sklearn import tree
    clf = tree.DecisionTreeClassifier()
    return clf

In [0]:
def Model(X_train, Y_train, X_test, Y_test):
  from sklearn.svm import SVC
  svm = SVC(kernel='linear',probability=True)
  # mlp = train_mlp(X_train, Y_train)
  logistic_regression = train_logistic_regression()
  j_48 = train_j48()

  majority_voting = VotingClassifier(estimators=[('svm', svm), ('j48', j_48), ('lr', logistic_regression)], voting='soft')
  majority_voting.fit(X_train, Y_train)
  Y_pred = majority_voting.predict(X_test)

  acc = accuracy_score(Y_test, Y_pred)
  recall = recall_score(Y_test, Y_pred, average='macro')
  f1 = f1_score(Y_test, Y_pred, average='macro')

  return acc,recall,f1


In [0]:
def DataPreparation(data_input_file):
  print('Catal et al. 2015 with SVM {}'.format(data_input_file))
  data = np.load(data_input_file)
  X = data['X']
  X = X[:, 0, :, :]
  Y = data['y']
  folds = data['folds']
  classes_number = Y.shape[1]
  Y = np.argmax(Y, axis=1)
  return X,Y,folds

In [0]:
def Train(X,Y,folds):
  avg_acc = []
  avg_recall = []
  avg_f1 = []
  for i in range(0, len(folds)):
          train_idx = folds[i][0]
          test_idx = folds[i][1]

          X_train = X[train_idx]
          X_test = X[test_idx]

          X_train = feature_extraction(X_train)
          X_test = feature_extraction(X_test)

          acc_fold, recall_fold, f1_fold=Model(X_train, Y[train_idx], X_test,Y[test_idx])

          avg_acc.append(acc_fold)
          avg_recall.append(recall_fold)
          avg_f1.append(f1_fold)

          print('Accuracy[{:.4f}] Recall[{:.4f}] F1[{:.4f}] at fold[{}]'.format(acc_fold, recall_fold, f1_fold ,i+1))
          print('________________________________________________________________')
    
  return avg_acc, avg_recall,avg_f1 

In [0]:
def ReportAccuracies(avg_acc, avg_recall,avg_f1):
  ic_acc = st.t.interval(0.9, len(avg_acc) - 1, loc=np.mean(avg_acc), scale=st.sem(avg_acc))
  ic_recall = st.t.interval(0.9, len(avg_recall) - 1, loc=np.mean(avg_recall), scale=st.sem(avg_recall))
  ic_f1 = st.t.interval(0.9, len(avg_f1) - 1, loc=np.mean(avg_f1), scale=st.sem(avg_f1))
  print('Mean Accuracy[{:.4f}] IC [{:.4f}, {:.4f}]'.format(np.mean(avg_acc), ic_acc[0], ic_acc[1]))
  print('Mean Recall[{:.4f}] IC [{:.4f}, {:.4f}]'.format(np.mean(avg_recall), ic_recall[0], ic_recall[1]))
  print('Mean F1[{:.4f}] IC [{:.4f}, {:.4f}]'.format(np.mean(avg_f1), ic_f1[0], ic_f1[1]))

In [0]:
def Run(data_input_file):
  X,Y,folds= DataPreparation(data_input_file)
  avg_acc, avg_recall,avg_f1= Train(X,Y,folds)
  ReportAccuracies(avg_acc, avg_recall,avg_f1 )

In [0]:
Run('data/LOSO/MHEALTH.npz')

Catal et al. 2015 with SVM data/LOSO/MHEALTH.npz
Accuracy[1.0000] Recall[1.0000] F1[1.0000] at fold[1]
________________________________________________________________
Accuracy[0.9620] Recall[0.9679] F1[0.9574] at fold[2]
________________________________________________________________
Accuracy[0.9960] Recall[0.9964] F1[0.9962] at fold[3]
________________________________________________________________
Accuracy[0.9173] Recall[0.9120] F1[0.8957] at fold[4]
________________________________________________________________
Accuracy[0.8679] Recall[0.8732] F1[0.8422] at fold[5]
________________________________________________________________
Accuracy[0.9205] Recall[0.9239] F1[0.9039] at fold[6]
________________________________________________________________
Accuracy[1.0000] Recall[1.0000] F1[1.0000] at fold[7]
________________________________________________________________
Accuracy[1.0000] Recall[1.0000] F1[1.0000] at fold[8]
________________________________________________________________

In [0]:
Run('data/LOTO/MHEALTH.npz')

Catal et al. 2015 with SVM data/LOTO/MHEALTH.npz
Accuracy[0.8867] Recall[0.8866] F1[0.8667] at fold[1]
________________________________________________________________
Accuracy[1.0000] Recall[1.0000] F1[1.0000] at fold[2]
________________________________________________________________
Accuracy[0.7985] Recall[0.8102] F1[0.7646] at fold[3]
________________________________________________________________
Accuracy[0.9686] Recall[0.9710] F1[0.9708] at fold[4]
________________________________________________________________
Accuracy[0.9544] Recall[0.9565] F1[0.9533] at fold[5]
________________________________________________________________
Accuracy[0.9964] Recall[0.9975] F1[0.9972] at fold[6]
________________________________________________________________
Accuracy[0.9192] Recall[0.9239] F1[0.9039] at fold[7]
________________________________________________________________
Accuracy[1.0000] Recall[1.0000] F1[1.0000] at fold[8]
________________________________________________________________

In [0]:
Run('data/SNOW/MHEALTH.npz')

Catal et al. 2015 with SVM data/SNOW/MHEALTH.npz
Accuracy[1.0000] Recall[1.0000] F1[1.0000] at fold[1]
________________________________________________________________
Accuracy[1.0000] Recall[1.0000] F1[1.0000] at fold[2]
________________________________________________________________
Accuracy[1.0000] Recall[1.0000] F1[1.0000] at fold[3]
________________________________________________________________
Accuracy[1.0000] Recall[1.0000] F1[1.0000] at fold[4]
________________________________________________________________
Accuracy[1.0000] Recall[1.0000] F1[1.0000] at fold[5]
________________________________________________________________
Accuracy[1.0000] Recall[1.0000] F1[1.0000] at fold[6]
________________________________________________________________
Accuracy[1.0000] Recall[1.0000] F1[1.0000] at fold[7]
________________________________________________________________
Accuracy[1.0000] Recall[1.0000] F1[1.0000] at fold[8]
________________________________________________________________

In [0]:
Run('data/FNOW/MHEALTH.npz')

Catal et al. 2015 with SVM data/FNOW/MHEALTH.npz
Accuracy[1.0000] Recall[1.0000] F1[1.0000] at fold[1]
________________________________________________________________
Accuracy[1.0000] Recall[1.0000] F1[1.0000] at fold[2]
________________________________________________________________
Accuracy[1.0000] Recall[1.0000] F1[1.0000] at fold[3]
________________________________________________________________
Accuracy[1.0000] Recall[1.0000] F1[1.0000] at fold[4]
________________________________________________________________
Accuracy[1.0000] Recall[1.0000] F1[1.0000] at fold[5]
________________________________________________________________
Accuracy[1.0000] Recall[1.0000] F1[1.0000] at fold[6]
________________________________________________________________
Accuracy[0.9850] Recall[0.9861] F1[0.9860] at fold[7]
________________________________________________________________
Accuracy[0.9925] Recall[0.9931] F1[0.9930] at fold[8]
________________________________________________________________

In [0]:
Run('data/LOSO/USCHAD.npz')

Catal et al. 2015 with SVM data/LOSO/USCHAD.npz
Accuracy[0.7727] Recall[0.7503] F1[0.7001] at fold[1]
________________________________________________________________
Accuracy[0.6228] Recall[0.5573] F1[0.5275] at fold[2]
________________________________________________________________
Accuracy[0.8297] Recall[0.7839] F1[0.7741] at fold[3]
________________________________________________________________
Accuracy[0.7866] Recall[0.6984] F1[0.6779] at fold[4]
________________________________________________________________
Accuracy[0.5778] Recall[0.4995] F1[0.4624] at fold[5]
________________________________________________________________
Accuracy[0.7843] Recall[0.7483] F1[0.7313] at fold[6]
________________________________________________________________
Accuracy[0.6435] Recall[0.6491] F1[0.6082] at fold[7]
________________________________________________________________
Accuracy[0.8420] Recall[0.7874] F1[0.7620] at fold[8]
________________________________________________________________


In [0]:
Run('data/LOTO/USCHAD.npz')

Catal et al. 2015 with SVM data/LOTO/USCHAD.npz
Accuracy[0.8413] Recall[0.8132] F1[0.8161] at fold[1]
________________________________________________________________
Accuracy[0.8594] Recall[0.8221] F1[0.8252] at fold[2]
________________________________________________________________
Accuracy[0.9013] Recall[0.8813] F1[0.8776] at fold[3]
________________________________________________________________
Accuracy[0.8536] Recall[0.8209] F1[0.8258] at fold[4]
________________________________________________________________
Accuracy[0.8768] Recall[0.8348] F1[0.8293] at fold[5]
________________________________________________________________
Accuracy[0.8938] Recall[0.8513] F1[0.8510] at fold[6]
________________________________________________________________
Accuracy[0.8842] Recall[0.8487] F1[0.8497] at fold[7]
________________________________________________________________
Accuracy[0.8924] Recall[0.8387] F1[0.8421] at fold[8]
________________________________________________________________


In [0]:
Run('data/SNOW/USCHAD.npz')

Catal et al. 2015 with SVM data/SNOW/USCHAD.npz
Accuracy[0.9140] Recall[0.8920] F1[0.8911] at fold[1]
________________________________________________________________
Accuracy[0.8947] Recall[0.8617] F1[0.8646] at fold[2]
________________________________________________________________
Accuracy[0.9016] Recall[0.8756] F1[0.8771] at fold[3]
________________________________________________________________
Accuracy[0.8995] Recall[0.8775] F1[0.8771] at fold[4]
________________________________________________________________
Accuracy[0.9103] Recall[0.8834] F1[0.8855] at fold[5]
________________________________________________________________
Accuracy[0.9194] Recall[0.8960] F1[0.8986] at fold[6]
________________________________________________________________
Accuracy[0.9234] Recall[0.9018] F1[0.9040] at fold[7]
________________________________________________________________
Accuracy[0.9030] Recall[0.8721] F1[0.8741] at fold[8]
________________________________________________________________


In [0]:
Run('data/FNOW/USCHAD.npz')

Catal et al. 2015 with SVM data/FNOW/USCHAD.npz
Accuracy[0.8841] Recall[0.8535] F1[0.8539] at fold[1]
________________________________________________________________
Accuracy[0.8917] Recall[0.8658] F1[0.8665] at fold[2]
________________________________________________________________
Accuracy[0.8812] Recall[0.8502] F1[0.8518] at fold[3]
________________________________________________________________
Accuracy[0.8968] Recall[0.8734] F1[0.8734] at fold[4]
________________________________________________________________
Accuracy[0.9048] Recall[0.8761] F1[0.8794] at fold[5]
________________________________________________________________
Accuracy[0.8968] Recall[0.8746] F1[0.8751] at fold[6]
________________________________________________________________
Accuracy[0.8767] Recall[0.8455] F1[0.8492] at fold[7]
________________________________________________________________
Accuracy[0.8982] Recall[0.8687] F1[0.8691] at fold[8]
________________________________________________________________


In [0]:
Run('data/LOSO/UTD-MHAD1_1s.npz')

Catal et al. 2015 with SVM data/LOSO/UTD-MHAD1_1s.npz
Accuracy[0.3837] Recall[0.3775] F1[0.3761] at fold[1]
________________________________________________________________
Accuracy[0.3795] Recall[0.3787] F1[0.3586] at fold[2]
________________________________________________________________
Accuracy[0.3251] Recall[0.3140] F1[0.3151] at fold[3]
________________________________________________________________
Accuracy[0.3237] Recall[0.3118] F1[0.3080] at fold[4]
________________________________________________________________
Accuracy[0.3079] Recall[0.3124] F1[0.2980] at fold[5]
________________________________________________________________
Accuracy[0.3705] Recall[0.3694] F1[0.3569] at fold[6]
________________________________________________________________
Accuracy[0.3278] Recall[0.3231] F1[0.3230] at fold[7]
________________________________________________________________
Accuracy[0.2993] Recall[0.3095] F1[0.2839] at fold[8]
___________________________________________________________

In [0]:
Run('data/LOTO/UTD-MHAD1_1s.npz')

Catal et al. 2015 with SVM data/LOTO/UTD-MHAD1_1s.npz
Accuracy[0.5248] Recall[0.5155] F1[0.5121] at fold[1]
________________________________________________________________
Accuracy[0.4958] Recall[0.4955] F1[0.4865] at fold[2]
________________________________________________________________
Accuracy[0.4540] Recall[0.4625] F1[0.4514] at fold[3]
________________________________________________________________


In [0]:
Run('data/SNOW/UTD-MHAD1_1s.npz')

Catal et al. 2015 with SVM data/SNOW/UTD-MHAD1_1s.npz
Accuracy[0.5481] Recall[0.5459] F1[0.5417] at fold[1]
________________________________________________________________
Accuracy[0.4319] Recall[0.4316] F1[0.4320] at fold[2]
________________________________________________________________
Accuracy[0.5407] Recall[0.5300] F1[0.5282] at fold[3]
________________________________________________________________
Accuracy[0.4698] Recall[0.4642] F1[0.4619] at fold[4]
________________________________________________________________
Accuracy[0.5237] Recall[0.5180] F1[0.5134] at fold[5]
________________________________________________________________
Accuracy[0.5040] Recall[0.5036] F1[0.4970] at fold[6]
________________________________________________________________
Accuracy[0.5027] Recall[0.4978] F1[0.4879] at fold[7]
________________________________________________________________


In [0]:
Run('data/FNOW/UTD-MHAD1_1s.npz')

Catal et al. 2015 with SVM data/FNOW/UTD-MHAD1_1s.npz
Accuracy[0.4742] Recall[0.4727] F1[0.4705] at fold[1]
________________________________________________________________
Accuracy[0.4977] Recall[0.4986] F1[0.5006] at fold[2]
________________________________________________________________
Accuracy[0.4619] Recall[0.4581] F1[0.4520] at fold[3]
________________________________________________________________
Accuracy[0.4567] Recall[0.4630] F1[0.4506] at fold[4]
________________________________________________________________
Accuracy[0.4709] Recall[0.4608] F1[0.4564] at fold[5]
________________________________________________________________
Accuracy[0.5097] Recall[0.5064] F1[0.5040] at fold[6]
________________________________________________________________
Accuracy[0.5149] Recall[0.5182] F1[0.5187] at fold[7]
________________________________________________________________
Accuracy[0.5126] Recall[0.5078] F1[0.5092] at fold[8]
___________________________________________________________

In [0]:
Run('data/LOSO/UTD-MHAD2_1s.npz')

Catal et al. 2015 with SVM data/LOSO/UTD-MHAD2_1s.npz
Accuracy[0.7933] Recall[0.7734] F1[0.7845] at fold[1]
________________________________________________________________
Accuracy[0.8182] Recall[0.7934] F1[0.7995] at fold[2]
________________________________________________________________
Accuracy[0.7434] Recall[0.7619] F1[0.7382] at fold[3]
________________________________________________________________
Accuracy[0.8943] Recall[0.8997] F1[0.8943] at fold[4]
________________________________________________________________
Accuracy[0.7516] Recall[0.7402] F1[0.7532] at fold[5]
________________________________________________________________
Accuracy[0.7443] Recall[0.7386] F1[0.7332] at fold[6]
________________________________________________________________
Accuracy[0.7857] Recall[0.7873] F1[0.7879] at fold[7]
________________________________________________________________
Accuracy[0.4626] Recall[0.4449] F1[0.4290] at fold[8]
___________________________________________________________

In [0]:
Run('data/LOTO/UTD-MHAD2_1s.npz')

Catal et al. 2015 with SVM data/LOTO/UTD-MHAD2_1s.npz
Accuracy[0.7628] Recall[0.7815] F1[0.7685] at fold[1]
________________________________________________________________
Accuracy[0.8281] Recall[0.8209] F1[0.8198] at fold[2]
________________________________________________________________
Accuracy[0.7925] Recall[0.7911] F1[0.7984] at fold[3]
________________________________________________________________
Accuracy[0.9100] Recall[0.9066] F1[0.9019] at fold[4]
________________________________________________________________
Accuracy[0.8230] Recall[0.8333] F1[0.8123] at fold[5]
________________________________________________________________
Accuracy[0.8455] Recall[0.8513] F1[0.8444] at fold[6]
________________________________________________________________
Accuracy[0.7679] Recall[0.7786] F1[0.7722] at fold[7]
________________________________________________________________
Accuracy[0.8700] Recall[0.8606] F1[0.8619] at fold[8]
___________________________________________________________

In [0]:
Run('data/SNOW/UTD-MHAD2_1s.npz')

Catal et al. 2015 with SVM data/SNOW/UTD-MHAD2_1s.npz
Accuracy[0.8220] Recall[0.8140] F1[0.8123] at fold[1]
________________________________________________________________
Accuracy[0.7627] Recall[0.7685] F1[0.7570] at fold[2]
________________________________________________________________
Accuracy[0.8190] Recall[0.8049] F1[0.8188] at fold[3]
________________________________________________________________
Accuracy[0.8407] Recall[0.8394] F1[0.8375] at fold[4]
________________________________________________________________
Accuracy[0.8304] Recall[0.8325] F1[0.8278] at fold[5]
________________________________________________________________
Accuracy[0.8482] Recall[0.8414] F1[0.8474] at fold[6]
________________________________________________________________
Accuracy[0.8839] Recall[0.8756] F1[0.8786] at fold[7]
________________________________________________________________
Accuracy[0.8036] Recall[0.7965] F1[0.7963] at fold[8]
___________________________________________________________

In [0]:
Run('data/FNOW/UTD-MHAD2_1s.npz')

Catal et al. 2015 with SVM data/FNOW/UTD-MHAD2_1s.npz
Accuracy[0.8095] Recall[0.8047] F1[0.8114] at fold[1]
________________________________________________________________
Accuracy[0.8254] Recall[0.8302] F1[0.8308] at fold[2]
________________________________________________________________
Accuracy[0.8095] Recall[0.8031] F1[0.7964] at fold[3]
________________________________________________________________
Accuracy[0.7778] Recall[0.7652] F1[0.7782] at fold[4]
________________________________________________________________
Accuracy[0.8197] Recall[0.8180] F1[0.8207] at fold[5]
________________________________________________________________
Accuracy[0.8033] Recall[0.7970] F1[0.8005] at fold[6]
________________________________________________________________
Accuracy[0.8197] Recall[0.8112] F1[0.8065] at fold[7]
________________________________________________________________
Accuracy[0.7869] Recall[0.7867] F1[0.7867] at fold[8]
___________________________________________________________

In [0]:
Run('data/LOTO/WHARF.npz')

Catal et al. 2015 with SVM data/LOTO/WHARF.npz
Accuracy[0.6617] Recall[0.5949] F1[0.5973] at fold[1]
________________________________________________________________
Accuracy[0.6602] Recall[0.5515] F1[0.5609] at fold[2]
________________________________________________________________
Accuracy[0.6390] Recall[0.5643] F1[0.5563] at fold[3]
________________________________________________________________
Accuracy[0.6734] Recall[0.6178] F1[0.6129] at fold[4]
________________________________________________________________
Accuracy[0.6204] Recall[0.5875] F1[0.5262] at fold[5]
________________________________________________________________
Accuracy[0.6556] Recall[0.5829] F1[0.5794] at fold[6]
________________________________________________________________
Accuracy[0.6185] Recall[0.5606] F1[0.5522] at fold[7]
________________________________________________________________
Accuracy[0.6384] Recall[0.5592] F1[0.5607] at fold[8]
________________________________________________________________
A

In [0]:
Run('data/LOSO/WHARF.npz')

Catal et al. 2015 with SVM data/LOSO/WHARF.npz
Accuracy[0.5553] Recall[0.4502] F1[0.4271] at fold[1]
________________________________________________________________
Accuracy[0.4706] Recall[0.2533] F1[0.2647] at fold[2]
________________________________________________________________
Accuracy[0.3652] Recall[0.2169] F1[0.2199] at fold[3]
________________________________________________________________
Accuracy[0.3952] Recall[0.2150] F1[0.1941] at fold[4]
________________________________________________________________
Accuracy[0.8571] Recall[0.7000] F1[0.7222] at fold[5]
________________________________________________________________
Accuracy[0.5829] Recall[0.4830] F1[0.4563] at fold[6]
________________________________________________________________
Accuracy[0.5379] Recall[0.5361] F1[0.4827] at fold[7]
________________________________________________________________
Accuracy[0.4476] Recall[0.2416] F1[0.2439] at fold[8]
________________________________________________________________
A

In [0]:
Run('data/SNOW/WHARF.npz')

Catal et al. 2015 with SVM data/SNOW/WHARF.npz
Accuracy[0.7038] Recall[0.6196] F1[0.6104] at fold[1]
________________________________________________________________
Accuracy[0.6752] Recall[0.6047] F1[0.6066] at fold[2]
________________________________________________________________
Accuracy[0.6905] Recall[0.5987] F1[0.6153] at fold[3]
________________________________________________________________
Accuracy[0.6941] Recall[0.5828] F1[0.5941] at fold[4]
________________________________________________________________
Accuracy[0.6992] Recall[0.6380] F1[0.6375] at fold[5]
________________________________________________________________
Accuracy[0.6770] Recall[0.6288] F1[0.6288] at fold[6]
________________________________________________________________
Accuracy[0.6770] Recall[0.6014] F1[0.5908] at fold[7]
________________________________________________________________
Accuracy[0.6805] Recall[0.6211] F1[0.6219] at fold[8]
________________________________________________________________
A

In [0]:
Run('data/FNOW/WHARF.npz')

Catal et al. 2015 with SVM data/FNOW/WHARF.npz
Accuracy[0.5818] Recall[0.5118] F1[0.5158] at fold[1]
________________________________________________________________
Accuracy[0.6301] Recall[0.5615] F1[0.5675] at fold[2]
________________________________________________________________
Accuracy[0.5936] Recall[0.4978] F1[0.4993] at fold[3]
________________________________________________________________
Accuracy[0.6313] Recall[0.5661] F1[0.5723] at fold[4]
________________________________________________________________
Accuracy[0.6175] Recall[0.5625] F1[0.5523] at fold[5]
________________________________________________________________
Accuracy[0.6432] Recall[0.5681] F1[0.5736] at fold[6]
________________________________________________________________
Accuracy[0.6509] Recall[0.5984] F1[0.5994] at fold[7]
________________________________________________________________
Accuracy[0.6161] Recall[0.5388] F1[0.5369] at fold[8]
________________________________________________________________
A

In [0]:
Run('data/LOTO/WISDM.npz')

Catal et al. 2015 with SVM data/LOTO/WISDM.npz
Accuracy[0.8194] Recall[0.7866] F1[0.7877] at fold[1]
________________________________________________________________
Accuracy[0.7970] Recall[0.6216] F1[0.5990] at fold[2]
________________________________________________________________
Accuracy[0.8439] Recall[0.7056] F1[0.7215] at fold[3]
________________________________________________________________
Accuracy[0.8701] Recall[0.7975] F1[0.8137] at fold[4]
________________________________________________________________
Accuracy[0.8333] Recall[0.7901] F1[0.7888] at fold[5]
________________________________________________________________
Accuracy[0.7650] Recall[0.7566] F1[0.7502] at fold[6]
________________________________________________________________
Accuracy[0.7701] Recall[0.7958] F1[0.7814] at fold[7]
________________________________________________________________
Accuracy[0.8387] Recall[0.8267] F1[0.8292] at fold[8]
________________________________________________________________
A

In [0]:
Run('data/LOSO/WISDM.npz')

Catal et al. 2015 with SVM data/LOSO/WISDM.npz
Accuracy[0.8041] Recall[0.6481] F1[0.6441] at fold[1]
________________________________________________________________
Accuracy[0.6115] Recall[0.2656] F1[0.2828] at fold[2]
________________________________________________________________
Accuracy[0.7242] Recall[0.6858] F1[0.6762] at fold[3]
________________________________________________________________
Accuracy[0.8720] Recall[0.8455] F1[0.8424] at fold[4]
________________________________________________________________
Accuracy[0.7194] Recall[0.5102] F1[0.4575] at fold[5]
________________________________________________________________
Accuracy[0.9360] Recall[0.6447] F1[0.6767] at fold[6]
________________________________________________________________
Accuracy[0.7015] Recall[0.5758] F1[0.5723] at fold[7]
________________________________________________________________
Accuracy[0.4481] Recall[0.3858] F1[0.3971] at fold[8]
________________________________________________________________
A

In [0]:
Run('data/SNOW/WISDM.npz')

Catal et al. 2015 with SVM data/SNOW/WISDM.npz
Accuracy[0.9062] Recall[0.8484] F1[0.8666] at fold[1]
________________________________________________________________
Accuracy[0.9023] Recall[0.8479] F1[0.8674] at fold[2]
________________________________________________________________
Accuracy[0.8988] Recall[0.8438] F1[0.8646] at fold[3]
________________________________________________________________
Accuracy[0.8945] Recall[0.8317] F1[0.8522] at fold[4]
________________________________________________________________
Accuracy[0.8992] Recall[0.8440] F1[0.8649] at fold[5]
________________________________________________________________
Accuracy[0.9001] Recall[0.8492] F1[0.8654] at fold[6]
________________________________________________________________
Accuracy[0.8987] Recall[0.8432] F1[0.8619] at fold[7]
________________________________________________________________
Accuracy[0.9054] Recall[0.8492] F1[0.8670] at fold[8]
________________________________________________________________
A

In [0]:
Run('data/FNOW/WISDM.npz')

Catal et al. 2015 with SVM data/FNOW/WISDM.npz
Accuracy[0.8844] Recall[0.8205] F1[0.8382] at fold[1]
________________________________________________________________
Accuracy[0.8825] Recall[0.8337] F1[0.8519] at fold[2]
________________________________________________________________
Accuracy[0.8994] Recall[0.8520] F1[0.8666] at fold[3]
________________________________________________________________
Accuracy[0.8651] Recall[0.7928] F1[0.8071] at fold[4]
________________________________________________________________
Accuracy[0.8934] Recall[0.8358] F1[0.8536] at fold[5]
________________________________________________________________
Accuracy[0.8848] Recall[0.8230] F1[0.8406] at fold[6]
________________________________________________________________
Accuracy[0.8876] Recall[0.8310] F1[0.8460] at fold[7]
________________________________________________________________
Accuracy[0.8924] Recall[0.8384] F1[0.8562] at fold[8]
________________________________________________________________
A